# "Trial-wise" distance model analysis

Notebook for analysis requested by reviewers, which will model "trialwise" metrics of path efficiency, pattern similarity, and link distance. 

8/3/2022


In [1]:
#Imports 
import csv
import numpy as np
import math
import pandas as pd
import os.path
#matplotlib inline
import matplotlib.pyplot as plt

In [2]:
#read in trialwise csv
hpc_rois = ['HPC_left','HPC_right']
efficiency="/Users/corey/Dropbox/Project/analysis/mixedmodels/HPC_PAPER/Finals_github/trialwiseanalysis.csv"
df_pe =  pd.read_csv(efficiency)

#read in pattern similarity csv
similarity = "/Users/corey/Dropbox/Project/analysis/mixedmodels/psa_longform_allSubs_june2021.csv"
df_sim = pd.read_csv(similarity)
df_ps = df_sim.loc[(df_sim.sesNum == 2)]

#read in trialwise model csv (so we dont need to rerun main loop)
model = "/Users/corey/Dropbox/Project/analysis/mixedmodels/HPC_PAPER/Finals_github/trialwiseModel.csv"
df_model = pd.read_csv(model)

In [3]:
# make a bunch of lists that will be populated later
neuralsimilarity_HPC_bilateral = []
neuralsimilarity_HPC_left = []
neuralsimilarity_HPC_right = []

stim1act_HPC_bilateral = []
stim1act_HPC_left = []
stim1act_HPC_right = []

stim2act_HPC_bilateral = []
stim2act_HPC_left = []
stim2act_HPC_right = []

v1_HPC_bilateral = []
v1_HPC_left = []
v1_HPC_right = []

it_HPC_bilateral = []
it_HPC_left = []
it_HPC_right = []

# loop through rows of the behavioral df
for i,row in df_pe.iterrows():
    # get info needed from pe df
    subNum = row['subID']
    startlandmark = str(row['startlandmark'])
    endlandmark = str(row['landmarkID'])
    
    # get info needed from ps df
    value1 = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startlandmark) & (df_ps.stim2_ID == endlandmark),
                   'fisher_r'].values[0]
    value2 = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum)
                   & (df_ps.stim2_ID == startlandmark) & (df_ps.stim1_ID == endlandmark),
                   'fisher_r'].values[0]
    value3 = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum)
                   & (df_ps.stim1_ID == startlandmark) & (df_ps.stim2_ID == endlandmark),
                   'fisher_r'].values[0]
    value4 = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum)
                   & (df_ps.stim2_ID == startlandmark) & (df_ps.stim1_ID == endlandmark),
                   'fisher_r'].values[0]
    # average the similarity values from matrix, and from both hemispheres
    ps = (value1 + value2 + value3 + value4)/4
    ps_right = (value1 + value2)/2
    ps_left = (value3 + value4)/2
    neuralsimilarity_HPC_bilateral.append(ps)
    neuralsimilarity_HPC_left.append(ps_left)
    neuralsimilarity_HPC_right.append(ps_right)
    
    # also get avg activation for the stims and v1/IT coregressor info
    stim1_left = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startlandmark) & (df_ps.stim2_ID == endlandmark),
                   'stim1_avgAct'].values[0]
    stim1_right = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startlandmark) & (df_ps.stim2_ID == endlandmark),
                   'stim1_avgAct'].values[0]
    stim1 = (stim1_left + stim1_right)/2
    stim1act_HPC_bilateral.append(stim1)
    stim1act_HPC_left.append(stim1_left)
    stim1act_HPC_right.append(stim1_right)
    
    stim2_left = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startlandmark) & (df_ps.stim2_ID == endlandmark),
                   'stim2_avgAct'].values[0]
    stim2_right = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startlandmark) & (df_ps.stim2_ID == endlandmark),
                   'stim2_avgAct'].values[0]
    stim2 = (stim2_left + stim2_right)/2
    stim2act_HPC_bilateral.append(stim2)
    stim2act_HPC_left.append(stim2_left)
    stim2act_HPC_right.append(stim2_right)
    
    v1_left = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startlandmark) & (df_ps.stim2_ID == endlandmark),
                   'v1_similarity'].values[0]
    v1_right = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startlandmark) & (df_ps.stim2_ID == endlandmark),
                   'v1_similarity'].values[0]
    v1 = (v1_left + v1_right)/2
    v1_HPC_bilateral.append(v1)
    v1_HPC_left.append(v1_left)
    v1_HPC_right.append(v1_right)
    
    it_left = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startlandmark) & (df_ps.stim2_ID == endlandmark),
                   'IT_similarity'].values[0]
    it_right = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startlandmark) & (df_ps.stim2_ID == endlandmark),
                   'IT_similarity'].values[0]
    it = (it_left + it_right)/2
    it_HPC_bilateral.append(it)
    it_HPC_left.append(it_left)
    it_HPC_right.append(it_right)

In [4]:
df_pe['neuralSimilarity_HPC_bilateral'] = neuralsimilarity_HPC_bilateral
df_pe['stim1act_HPC_bilateral'] = stim1act_HPC_bilateral
df_pe['stim2act_HPC_bilateral'] = stim2act_HPC_bilateral
df_pe['V1_HPC_bilateral'] = v1_HPC_bilateral
df_pe['IT_HPC_bilateral'] = it_HPC_bilateral
# left
df_pe['neuralSimilarity_HPC_left'] = neuralsimilarity_HPC_left
df_pe['stim1act_HPC_left'] = stim1act_HPC_left
df_pe['stim2act_HPC_left'] = stim2act_HPC_left
df_pe['V1_HPC_left'] = v1_HPC_left
df_pe['IT_HPC_left'] = it_HPC_left
# right
df_pe['neuralSimilarity_HPC_right'] = neuralsimilarity_HPC_right
df_pe['stim1act_HPC_right'] = stim1act_HPC_right
df_pe['stim2act_HPC_right'] = stim2act_HPC_right
df_pe['V1_HPC_right'] = v1_HPC_right
df_pe['IT_HPC_right'] = it_HPC_right

df_pe_filename = '/Users/corey/Dropbox/Project/analysis/mixedmodels/HPC_PAPER/Finals_github/trialwiseModel.csv'
df_pe.to_csv(df_pe_filename, encoding='utf-8', index = True)

In [3]:
# make a bunch of lists that will be populated later
neuralsimilarity_HPC_bilateral_fr = []
stim1act_HPC_bilateral_fr = []
stim2act_HPC_bilateral_fr = []
v1_HPC_bilateral_fr = []
it_HPC_bilateral_fr = []

# loop through rows of the behavioral df
for i,row in df_model.iterrows():
    # get info needed from pe df
    subNum = row['subID']
    startfractal = str(row['startfractal'])
    endfractal = str(row['endfractal'])
    
    # get info needed from ps df
    value1 = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startfractal) & (df_ps.stim2_ID == endfractal),
                   'fisher_r'].values[0]
    value2 = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum)
                   & (df_ps.stim2_ID == startfractal) & (df_ps.stim1_ID == endfractal),
                   'fisher_r'].values[0]
    value3 = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum)
                   & (df_ps.stim1_ID == startfractal) & (df_ps.stim2_ID == endfractal),
                   'fisher_r'].values[0]
    value4 = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum)
                   & (df_ps.stim2_ID == startfractal) & (df_ps.stim1_ID == endfractal),
                   'fisher_r'].values[0]
    # average the similarity values from matrix, and from both hemispheres
    ps = (value1 + value2 + value3 + value4)/4
    ps_right = (value1 + value2)/2
    ps_left = (value3 + value4)/2
    neuralsimilarity_HPC_bilateral_fr.append(ps)
    
    # also get avg activation for the stims and v1/IT coregressor info
    stim1_left = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startfractal) & (df_ps.stim2_ID == endfractal),
                   'stim1_avgAct'].values[0]
    stim1_right = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startfractal) & (df_ps.stim2_ID == endfractal),
                   'stim1_avgAct'].values[0]
    stim1 = (stim1_left + stim1_right)/2
    stim1act_HPC_bilateral_fr.append(stim1)
    
    stim2_left = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startfractal) & (df_ps.stim2_ID == endfractal),
                   'stim2_avgAct'].values[0]
    stim2_right = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startfractal) & (df_ps.stim2_ID == endfractal),
                   'stim2_avgAct'].values[0]
    stim2 = (stim2_left + stim2_right)/2
    stim2act_HPC_bilateral_fr.append(stim2)
    
    v1_left = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startfractal) & (df_ps.stim2_ID == endfractal),
                   'v1_similarity'].values[0]
    v1_right = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startfractal) & (df_ps.stim2_ID == endfractal),
                   'v1_similarity'].values[0]
    v1 = (v1_left + v1_right)/2
    v1_HPC_bilateral_fr.append(v1)
    
    it_left = df_ps.loc[(df_ps.roi == "HPC_left") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startfractal) & (df_ps.stim2_ID == endfractal),
                   'IT_similarity'].values[0]
    it_right = df_ps.loc[(df_ps.roi == "HPC_right") & (df_ps.subNum == subNum) 
                   & (df_ps.stim1_ID == startfractal) & (df_ps.stim2_ID == endfractal),
                   'IT_similarity'].values[0]
    it = (it_left + it_right)/2
    it_HPC_bilateral_fr.append(it)

In [4]:
df_model['neuralSimilarity_HPC_bilateral_fr'] = neuralsimilarity_HPC_bilateral_fr
df_model['stim1act_HPC_bilateral_fr'] = stim1act_HPC_bilateral_fr
df_model['stim2act_HPC_bilateral_fr'] = stim2act_HPC_bilateral_fr
df_model['V1_HPC_bilateral_fr'] = v1_HPC_bilateral_fr
df_model['IT_HPC_bilateral_fr'] = it_HPC_bilateral_fr

df_model_filename = '/Users/corey/Dropbox/Project/analysis/mixedmodels/HPC_PAPER/Finals_github/trialwiseModel.csv'
df_model.to_csv(df_model_filename, encoding='utf-8', index = True)